# Clasificador de Vinos con KNN (Wine Quality - Red)

**Objetivo:** predecir la calidad del vino (label) usando sus características químicas con **K-Nearest Neighbors**.

> Nota para el futuro: KNN depende de **distancias**, así que **escalar** las features es casi obligatorio para que ninguna columna domine el cálculo de distancia.



In [ ]:
# ==========================================
# Paso 0: Imports
# ==========================================

import pandas as pd  # Yo uso pandas para cargar y explorar datos.
import numpy as np   # Yo uso numpy para utilidades numéricas.
import matplotlib.pyplot as plt  # Yo uso matplotlib para graficar.


In [ ]:
# ==========================================
# Paso 1: Cargar datos y exploración rápida
# ==========================================

url = "https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/refs/heads/main/winequality-red.csv"  # Yo guardo el link del CSV.
df = pd.read_csv(url)  # Yo cargo el dataset.

print("Shape (filas, columnas):", df.shape)  # Yo reviso tamaño.
df.head()  # Yo miro primeras filas.


In [ ]:
# Se reviso la estructura (tipos, nulos, etc.).
df.info()

# Se obtengo estadísticos rápidos de las variables numéricas.
df.describe()


In [ ]:
# Se reviso si hay duplicados.
print("Duplicados:", df.duplicated().sum())

# (Opcional) Si quiero eliminarlos, lo haría así:
# df = df.drop_duplicates()


In [ ]:
# Se reviso valores nulos.
print("Nulos por columna:\n", df.isnull().sum())


In [ ]:
# Se reviso el balance de clases (label es mi target con 0/1/2).
print("Distribución de clases (label):\n", df["label"].value_counts())
print("\nProporción de clases (label):\n", df["label"].value_counts(normalize=True))


## Paso 2 — Preparación: X, y y train/test split (80/20)

> Nota para el futuro: uso **stratify=y** porque esto es clasificación multicategoría (0,1,2) y quiero mantener la proporción de clases en train/test.



In [ ]:
from sklearn.model_selection import train_test_split  # Yo importo la función para dividir los datos.

# Se separo features y target.
X = df.drop(columns=["label"])  # Yo tomo todas las columnas menos la etiqueta.
y = df["label"]                 # Yo tomo la columna objetivo.

# Se divido en train/test (80/20) y estratifico.
X_train, X_test, y_train, y_test = train_test_split(
    X,               # features
    y,               # target
    test_size=0.2,   # 20% para test
    random_state=42, # reproducibilidad
    stratify=y       # mantener proporción de clases
)

print("Train:", X_train.shape, "| Test:", X_test.shape)


## Paso 3 — Escalado de datos (recomendado en KNN)

> Nota para el futuro: **SIEMPRE** hago `fit` del scaler solo con **train** y luego `transform` a train y test, para evitar data leakage.



In [ ]:
from sklearn.preprocessing import StandardScaler  # Yo importo el escalador estándar.

scaler = StandardScaler()  # Yo inicializo el escalador.

# Se ajusto (fit) el scaler SOLO con X_train.
X_train_scaled = scaler.fit_transform(X_train)

# Se transformo X_test con el mismo scaler (sin re-ajustar).
X_test_scaled = scaler.transform(X_test)

print("Escalado listo. Shapes:", X_train_scaled.shape, X_test_scaled.shape)


## Paso 4 — Entrenar KNN con un k inicial y evaluar

Voy a empezar con un `k` inicial (por ejemplo 5) y luego lo optimizo probando varios k.



In [ ]:
from sklearn.neighbors import KNeighborsClassifier  # Yo importo el modelo KNN de clasificación.

# Se elijo un k inicial.
k_inicial = 5

# Se creo el modelo.
knn = KNeighborsClassifier(n_neighbors=k_inicial)

# Se entreno el modelo (en KNN esto básicamente guarda datos para comparar distancias).
knn.fit(X_train_scaled, y_train)

# Se predigo para test.
y_pred = knn.predict(X_test_scaled)

y_pred[:10]  # Yo miro algunas predicciones.


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report  # Yo importo métricas.

# Se calculo accuracy.
acc = accuracy_score(y_test, y_pred)
print("Accuracy (k =", k_inicial, "):", round(acc, 4))

# Se muestro matriz de confusión.
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion matrix:\n", cm)

# Se muestro reporte de clasificación (precision/recall/f1 por clase).
print("\nClassification report:\n", classification_report(y_test, y_pred))


## Paso 5 — Optimización de k (1 a 20)

Voy a probar k desde 1 hasta 20, guardar los accuracy y graficar `accuracy vs k` para elegir el mejor.



In [ ]:
k_values = list(range(1, 21))  # Yo defino la lista de K a probar.
accuracies = []                # Yo creo una lista para guardar accuracies.

for k in k_values:  # Yo itero sobre cada k.
    model = KNeighborsClassifier(n_neighbors=k)  # Yo creo el modelo con ese k.
    model.fit(X_train_scaled, y_train)           # Yo entreno con train escalado.
    pred = model.predict(X_test_scaled)          # Yo predigo en test escalado.
    accuracies.append(accuracy_score(y_test, pred))  # Yo guardo accuracy.

# Se reviso los resultados rápidamente.
results = pd.DataFrame({"k": k_values, "accuracy": accuracies})
results


In [ ]:
# Se encuentro el mejor k por accuracy.
best_idx = int(np.argmax(accuracies))  # índice del mayor accuracy
best_k = k_values[best_idx]            # k asociado
best_acc = accuracies[best_idx]        # accuracy máximo

print("Mejor k:", best_k)
print("Mejor accuracy:", round(best_acc, 4))


In [ ]:
# Se grafico accuracy vs k para ver el comportamiento.
plt.figure(figsize=(10, 5))
plt.plot(k_values, accuracies, marker="o")
plt.title("Accuracy vs k (KNN - Wine Quality Red)")
plt.xlabel("k (n_neighbors)")
plt.ylabel("Accuracy")
plt.xticks(k_values)
plt.tight_layout()
plt.show()


## Paso 6 — Entrenar modelo final con el mejor k y guardar (modelo + scaler)

> Nota para el futuro: debo guardar **también el scaler**, porque si no, al usar el modelo en producción no podré escalar datos nuevos igual.



In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay  # Yo importo display para matriz.
import joblib  # Yo uso joblib para guardar el modelo y el scaler.

# Se entreno el modelo final con best_k.
final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(X_train_scaled, y_train)

# Se evalúo una vez más el modelo final.
final_pred = final_model.predict(X_test_scaled)
final_acc = accuracy_score(y_test, final_pred)

print("Accuracy FINAL (k =", best_k, "):", round(final_acc, 4))
print("\nClassification report FINAL:\n", classification_report(y_test, final_pred))

# Se grafico matriz de confusión final.
ConfusionMatrixDisplay.from_predictions(y_test, final_pred)
plt.title(f"Matriz de confusión — KNN final (k={best_k})")
plt.tight_layout()
plt.show()

# Se guardo scaler y modelo.
joblib.dump(scaler, "wine_knn_scaler.joblib")
joblib.dump(final_model, f"wine_knn_model_k{best_k}.joblib")

print("Guardado:")
print("- wine_knn_scaler.joblib")
print(f"- wine_knn_model_k{best_k}.joblib")


## Checklist para entrega (para el futuro)

- ✅ Cargué el dataset y revisé info/describe/nulos/duplicados.
- ✅ Separé X / y.
- ✅ Hice train/test 80/20 con stratify.
- ✅ Escalé con StandardScaler (fit en train, transform en test).
- ✅ Entrené KNN con un k inicial y evalué con accuracy + confusion matrix + report.
- ✅ Probé k=1..20, guardé accuracies y grafiqué accuracy vs k.
- ✅ Entrené el modelo final con el mejor k y guardé modelo + scaler.

